In [14]:
import json
import openai
import os
import ast
OPENAI_KEY = os.environ['OPENAI_KEY']

client = openai.OpenAI(api_key=OPENAI_KEY)

In [15]:
f = open("/home/gabriel/projects/natural-plan/ground_truth_data/trip_planning.json",'r')
data = json.load(f)

In [16]:
k = "trip_planning_example_9"

In [17]:
ex = data[k]

In [18]:
print(ex['prompt_0shot'])

You plan to visit 3 European cities for 10 days in total. You only take direct flights to commute between cities. You want to spend 3 days in Frankfurt. You want to spend 3 days in Bucharest. You would like to visit Stuttgart for 6 days. You have to attend a workshop in Stuttgart between day 5 and day 10.

Here are the cities that have direct flights:
Bucharest and Frankfurt, Frankfurt and Stuttgart.

Find a trip plan of visiting the cities for 10 days by taking direct flights to commute between them.


In [19]:
print(ex['pred_5shot_pro'])

Here is the trip plan for visiting the 3 European cities for 10 days:

**Day 1-3:** Arriving in Bucharest and visit Bucharest for 3 days.
**Day 3:** Fly from Bucharest to Frankfurt.
**Day 3-5:** Visit Frankfurt for 3 days.
**Day 5:** Fly from Frankfurt to Stuttgart.
**Day 5-10:** Visit Stuttgart for 6 days. 



In [20]:
# task = ex['prompt_0shot']
# system = "You are an intelligent AI assistant."
# instruction = f"""
# We are creating a trip plan according to the following requirements.
# PROBLEM DESCRIPTION

# "{task}"

# END PROBLEM DESCRIPTION

# The proposed trip plan will be provided in the format
# SOLUTION FORMAT

# **Day x-y:** Visit <first city> for <y-x+1> days
# **Day y:** Fly from <first city> to <second city>
# **Day y-z:** Visit <second city> for <z-y+1> days
# ...

# END SOLUTION FORMAT

# We now need a python function that can parse the proposed solution and analyze whether all requirements and constraints of the task are met.
# * The function should check whether all cities are visited for the correct number of days.
# * Check whether the chosen flights are available
# * Check whether any constraints on visiting certain cities on specific days are met if the task requires it.
# * Note that the language used for arriving or flying might be variable, make the function robust to potential language changes
# * The function should return a list of strings indicating the constraints that are not met, or if all constraints are met

# Please create the python function, enclose the code in ```python tags
# """

In [21]:
task = ex['prompt_0shot']
system = "You are an intelligent AI assistant."
instruction = f"""
We are creating a trip plan according to the following requirements.
PROBLEM DESCRIPTION

"{task}"

END PROBLEM DESCRIPTION

We now need to extract a dictionary of constraints.
Produce a dictionary where the keys are the cities to be visited.
The values in the dictionary should be num_days: days the city needs to be visited, flights:list of flights available from that city, day_constraints:list of specific days the city needs to be visited.

Here is an example task description and corresponding output:
You want to visit european cities for 10 days. You want to visit Amsterdam for 3 days, Berlin for 5 days and Rome for 2 days. You plan to meet a friend in Rome on days 9-10 of the trip.
There are direct flights between Amsterdam and Berlin, Amsterdam and Rome.

Output:
{{
    "Amsterdam":{{"num_days":3, "flights":["Berlin", "Rome"], "day_constraints":[]}},
    "Berlin":{{"num_days":5, "flights":["Amsterdam"], "day_constraints":[]}},
    "Rome":{{"num_days":2, "flights":["Amsterdam"], "day_constraints":[9, 10]}}
}}

only output the dictionary, nothing else, do not enclose it in ```json tags
"""

In [22]:
response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": instruction}
        ],
        temperature=0.2,  # Adjust as needed
        max_tokens=4096,   # Adjust as needed
        seed=0
    )

resp = response.choices[0].message.content

In [23]:
print(resp)

{
    "Frankfurt": {"num_days": 3, "flights": ["Bucharest", "Stuttgart"], "day_constraints": []},
    "Bucharest": {"num_days": 3, "flights": ["Frankfurt"], "day_constraints": []},
    "Stuttgart": {"num_days": 6, "flights": ["Frankfurt"], "day_constraints": [5, 6, 7, 8, 9, 10]}
}


In [26]:
def parse_constraints(resp:str):
    if "```json" in resp:
        resp = resp.replace("```json\n","").replace("```","")

    if "```python" in resp:
        resp = resp.replace("```python\n","").replace("```","")

    try:
        return ast.literal_eval(resp)
    except:
        print(f"failed to parse {resp}")
        return None


In [27]:
d = parse_constraints(resp)

In [28]:
d

{'Frankfurt': {'num_days': 3,
  'flights': ['Bucharest', 'Stuttgart'],
  'day_constraints': []},
 'Bucharest': {'num_days': 3, 'flights': ['Frankfurt'], 'day_constraints': []},
 'Stuttgart': {'num_days': 6,
  'flights': ['Frankfurt'],
  'day_constraints': [5, 6, 7, 8, 9, 10]}}